<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы далеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Данные пришли с прокси-серверов Университета Блеза Паскаля и имеют очень простой вид. Для каждого пользователя заведен csv-файл с названием user\*\*\*\*.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: <br>

<center>*timestamp, посещенный веб-сайт*</center>

Скачать исходные данные можно по [ссылке](http://fc.isima.fr/~kahngi/cez13.zip) в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://yadi.sk/d/3gscKIdN3BCASG) на архив *capstone_user_identification* (~7 Mb, в развернутом виде ~ 60 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге `10users` лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога `150users` – там 150 файлов;
 - В `3users` – игрушечный пример из 3 файлов, это для отладки кода предобработки, который вы далее напишете.

На 5 неделе будет задание по [соревнованию](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle Inclass, которое организовано специально под Capstone проект нашей специализации. Соревнование уже открыто и, конечно, желающие могут начать уже сейчас.

# <center>Неделя 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы Scipy.sparse), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций 1 и 2 недели курса "Математика и Python для анализа данных":**
   - [Циклы, функции, генераторы, list comprehension](https://www.coursera.org/learn/mathematics-and-python/lecture/Kd7dL/tsikly-funktsii-ghienieratory-list-comprehension)
   - [Чтение данных из файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/8Xvwp/chtieniie-dannykh-iz-failov)
   - [Запись файлов, изменение файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/vde7k/zapis-failov-izmienieniie-failov)
   - [Pandas.DataFrame](https://www.coursera.org/learn/mathematics-and-python/lecture/rcjAW/pandas-data-frame)
   - [Pandas. Индексация и селекция](https://www.coursera.org/learn/mathematics-and-python/lecture/lsXAR/pandas-indieksatsiia-i-sieliektsiia)
   
**Кроме того, в задании будут использоваться библиотеки Python [glob](https://docs.python.org/3/library/glob.html), [pickle](https://docs.python.org/2/library/pickle.html) и класс [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html) из scipy.sparse.**

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark).

In [3]:
%load_ext watermark

In [4]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.8.0

numpy      : 1.19.5
scipy      : 1.3.1
pandas     : 1.3.2
matplotlib : 3.1.1
statsmodels: 0.13.0
sklearn    : 0.21.3

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 18.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit



In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm 
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = '/Users/zaharsaraev/Documents/Machine_learning_coursera/Final_project/'

In [7]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv'))

In [8]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

Детали:
- Смотрите чуть ниже пример вывода, что должна возвращать функция
- Используйте glob (или аналоги) для обхода файлов в каталоге. Для определенности, отсортируйте список файлов лексикографически. Удобно использовать `tqdm` для отслеживания числа выполненных итераций цикла
- Создайте частотный словарь уникальных сайтов (вида {'site_string': (site_id, site_freq)}) и заполняйте его по ходу чтения файлов. Начните с 1
- Рекомендуется меньшие индексы давать более часто попадающимся сайтам (приницип наименьшего описания)
- Не делайте entity recognition, считайте *google.com*, *http://www.google.com* и *www.google.com* разными сайтами (подключить entity recognition можно уже в рамках индивидуальной работы над проектом)
- Скорее всего в файле число записей не кратно числу *session_length*. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор [*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- В итоге некоторые сессии могут повторяться – оставьте как есть, не удаляйте дубликаты. Если в двух сессиях все сайты одинаковы, но сессии принадлежат разным пользователям, то тоже оставляйте как есть, это естественная неопределенность в данных
- Не оставляйте в частотном словаре сайт 0 (уже в конце, когда функция возвращает этот словарь)
- 150 файлов из *capstone_websites_data/150users/* у меня обработались за 1.7 секунды, но многое, конечно, зависит от реализации функции и от используемого железа. И вообще, первая реализация скорее всего будет не самой эффективной, дальше можно заняться профилированием (особенно если планируете запускать этот код для 3000 пользователей). Также эффективная реализация этой функции поможет нам на следующей неделе.

In [9]:
data = pd.read_csv('/Users/zaharsaraev/Documents/Machine_learning_coursera/Final_project/10users/user0031.csv')
data.site

0          fpdownload2.macromedia.com
1                         laposte.net
2                     www.laposte.net
3                      www.google.com
4                     www.laposte.net
                    ...              
7595              webmail.laposte.net
7596    av.mediapostcommunication.net
7597                   www.google.com
7598              webmail.laposte.net
7599    ws.mediapostcommunication.net
Name: site, Length: 7600, dtype: object

In [3]:
def get_id_by_key(d, key, id):
    for k, v in d.items():
        if k==key:
            return(v[0], id) #если сайт встречался в раньше, даем ему нужный ID, при этом значение для новых ID не меняется
    return(id+1, id+1) #если сайт раньше не встречался, то даем ему другой ID, и новые ID,
                            #будут меняться для следующих новых сайтов

In [4]:
def get_freq_by_key(d, key):
    for k, v in d.items():
        if key==k:
            return(v[1]+1)
    return(1)

In [5]:
def prepare_train_set(path_to_csv_files, session_length=10):
    columns=[]
    dict_sites = {} #Создаем словарь
    for i in range(1,session_length+1):
        columns.append('site{}'.format(i)) #Добавляем столбцы с сайтами
    columns.append('userID') #Добавляем столбец ID
    df = pd.DataFrame(columns=columns) #создаем пустой DataFrame
    new_id = 0 #ID начнем с единицы
    index = 0 #строки с единицы
    file_names = glob(path_to_csv_files+'/*')
    os.chdir(path_to_csv_files)
    for name in file_names:
        data = pd.read_csv(name) #Считываем файл
        #print(data)
        sites = np.array(data.site) #Достаем список сайтов
        start = 0
        flag = 0
        for j in range(len(sites)//session_length+1): #Здесь смотрим сколько отдельных сессий брать
            ids = [] #Создаем массив IDs сессии
            keys = []
            values = np.eye(2,10)
            for i in range(start, session_length+start):
                try: #Чтобы заполнить остатки нулями (см. инструкцию выше)
                    id_value, new_id = get_id_by_key(dict_sites, sites[i], new_id) #См. функцию выше
                    ids.append(id_value) #Добавляем ID
                    freq = get_freq_by_key(dict_sites, sites[i])
                    dict_sites.update({sites[i]: [id_value, freq]})
                
                except IndexError: 
                    if len(ids)==0: #Если выполняется случай полного деления, например 30//10, то будет лишняя строка с нулями
                        flag = 1 #Поэтому делаем break
                    else: #Иначе дозаполняем нулями
                        ids=list(np.hstack([ids, np.zeros(session_length-len(ids))]))
                        break
            userID = name
            userID = userID.replace(path_to_csv_files+'/user', '')
            userID = userID.replace('.csv', '')
            ids.append(userID)
            if flag==1:
                break
            df.loc[index]=ids
            index+=1
            start+=10
    return(df, dict_sites)

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [13]:
!cat $PATH_TO_DATA/3users/user0001.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,geo.mozilla.org
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,accounts.google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,apis.google.com
2013-11-15 11:41:35,plus.google.com
2013-11-15 12:40:35,vk.com
2013-11-15 12:40:37,google.com
2013-11-15 12:40:40,google.com
2013-11-15 12:41:35,google.com


In [14]:
!cat $PATH_TO_DATA/3users/user0002.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,football.kulichki.ru
2013-11-15 11:37:26,football.kulichki.ru
2013-11-15 11:40:32,oracle.com


In [15]:
!cat $PATH_TO_DATA/3users/user0003.csv

timestamp,site
2013-11-15 09:28:17,meduza.io
2013-11-15 09:33:04,google.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,google.com
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,yandex.ru
2013-11-15 11:41:35,meduza.io
2013-11-15 12:28:17,meduza.io
2013-11-15 12:33:04,google.com
2013-11-15 12:52:48,oracle.com


In [6]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

In [17]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,userID
0,1.0,2.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0002
1,4.0,5.0,2.0,5.0,2.0,5.0,5.0,6.0,7.0,4.0,0003
2,4.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0003
3,1.0,2.0,2.0,8.0,2.0,5.0,9.0,6.0,10.0,11.0,0001
4,1.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0001


Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

In [18]:
site_freq_3users

{'vk.com': [1, 3],
 'oracle.com': [2, 8],
 'football.kulichki.ru': [3, 2],
 'meduza.io': [4, 3],
 'google.com': [5, 9],
 'mail.google.com': [6, 2],
 'yandex.ru': [7, 1],
 'geo.mozilla.org': [8, 1],
 'accounts.google.com': [9, 1],
 'apis.google.com': [10, 1],
 'plus.google.com': [11, 1]}

**1. Примените полученную функцию к данным по 10 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к первому вопросу теста. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [19]:
%%time
train_data_10users, site_freq_10users = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 
                                                     session_length=10)

CPU times: user 59.3 s, sys: 584 ms, total: 59.9 s
Wall time: 1min 1s


In [20]:
train_data_10users

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,userID
0,1,2,2,3,4,4,4,5,5,6,0039
1,7,8,7,4,9,10,11,12,9,13,0039
2,14,15,15,9,15,9,9,4,4,9,0039
3,12,15,15,16,17,13,18,19,20,21,0039
4,22,23,19,21,24,19,25,26,21,19,0039
...,...,...,...,...,...,...,...,...,...,...,...
14056,60,538,538,60,761,52,538,60,761,538,0237
14057,60,761,52,538,761,60,538,761,761,60,0237
14058,538,60,761,538,60,16,59,52,538,60,0237
14059,538,16,60,538,538,761,52,59,761,761,0237


In [21]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [22]:
write_answer_to_file(len(train_data_10users), 
                     os.path.join(PATH_TO_DATA, 'answers/answer1_1.txt'))

**2. Сколько всего уникальных сайтов в выборке из 10 пользователей? ***


In [23]:
write_answer_to_file(len(site_freq_10users), 
                     os.path.join(PATH_TO_DATA, 'answers/answer1_2.txt'))

**3. Примените полученную функцию к данным по 150 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к третьему вопросу. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [27]:
%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)

CPU times: user 38min 34s, sys: 1min 21s, total: 39min 56s
Wall time: 40min 43s


In [30]:
write_answer_to_file(len(train_data_150users), 
                     os.path.join(PATH_TO_DATA, 'answers/answer1_3.txt'))

**4. Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [31]:
write_answer_to_file(len(site_freq_150users), 
                     os.path.join(PATH_TO_DATA, 'answers/answer1_4.txt'))

In [28]:
site_freq_150users

{'office14client.microsoft.com': [1, 259],
 'office.microsoft.com': [2, 699],
 'api.bing.com': [3, 4641],
 'rr.office.microsoft.com': [4, 48],
 'services.highbeam.com': [5, 27],
 'integrate.factiva.com': [6, 28],
 'www.worldlingo.com': [7, 26],
 'www.microsofttranslator.com': [8, 124],
 'www.bing.com': [9, 13797],
 'dev.virtualearth.net': [10, 112],
 'www.pagesjaunes.fr': [11, 540],
 'logc258.at.pagesjaunes.fr': [12, 128],
 'ts3.mm.bing.net': [13, 2740],
 'ts2.mm.bing.net': [14, 2759],
 'ts4.mm.bing.net': [15, 2746],
 'ts1.mm.bing.net': [16, 5380],
 'login.live.com': [17, 6726],
 'ssl.bing.com': [18, 1134],
 'translate.google.fr': [19, 5499],
 'www.google.com': [20, 51320],
 'www.google.fr': [21, 64785],
 'webmail.laposte.net': [22, 2293],
 'fpdownload2.macromedia.com': [23, 820],
 'clients1.google.fr': [24, 5758],
 'images.google.fr': [25, 10],
 'apis.google.com': [26, 29983],
 'cms.abmr.net': [27, 523],
 '1.perf.msedge.net': [28, 63],
 '2.perf.msedge.net': [29, 66],
 '3.perf.msedge.n

In [32]:
train_data_150users

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,userID
0,1,2,2,1,3,2,4,5,6,7,0158
1,8,2,9,9,10,9,11,12,13,14,0158
2,15,16,9,15,16,13,9,14,16,9,0158
3,9,16,9,9,9,17,9,9,14,13,0158
4,16,15,15,13,14,16,15,16,9,9,0158
...,...,...,...,...,...,...,...,...,...,...,...
137014,53,10465,10467,483,10463,31,10466,10467,41,672,0425
137015,59,31,826,33,232,31,20,21,61,38,0425
137016,9950,21,27797,27797,232,21,232,244,21,244,0425
137017,20,26,62,293,295,20,26,328,3825,72,0425


**5. Выведите топ-10 самых популярных сайтов среди посещенных 150 пользователями. Запишите ответ в файл *answer1_5.txt* через пробел – все 10 сайтов на одной строке.**

In [33]:
sort_dict = dict(sorted(site_freq_150users.items(), key=lambda item: item[1][1], reverse=True))
top10_popular = [] 
flag = 0
for k, v in sort_dict.items():
    top10_popular.append(k)
    flag+=1
    if flag==10:
        break

In [34]:
top10_popular

['www.google.fr',
 'www.google.com',
 'www.facebook.com',
 'apis.google.com',
 's.youtube.com',
 'clients1.google.com',
 'mail.google.com',
 'plus.google.com',
 'safebrowsing-cache.google.com',
 'www.youtube.com']

In [35]:
def write_list_answer_to_file(list, name):
    with open(name, "w") as fout:
        fout.write(" ".join([str(el) for el in list]))

In [36]:
write_list_answer_to_file(top10_popular, os.path.join(PATH_TO_DATA, 'answers/answer1_5.txt'))

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [37]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

**Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. **

**Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.** 

In [10]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [19]:
X_toy

array([[ 1.,  2.,  3.,  3.,  2.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  5.,  2.,  5.,  2.,  5.,  5.,  6.,  7.,  4.],
       [ 4.,  5.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  2.,  2.,  8.,  2.,  5.,  9.,  6., 10., 11.],
       [ 1.,  5.,  5.,  5.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [17]:
def csr(X):
    docs = X
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:
            if term == 0:
                continue
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))

    return(csr_matrix((data, indices, indptr), dtype=int), csr_matrix((data, indices, indptr), dtype=int).toarray())

In [15]:
def csr_new(X, voc):
    docs = X
    indptr = [0]
    indices = []
    data = []
    vocabulary = voc
    for d in docs:
        for term in d:
            if term == 0:
                continue
            index = vocabulary.setdefault(term, term)
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))

    return(csr_matrix((data, indices, indptr), dtype=int).toarray())

In [21]:
docs = X_toy
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in docs:
    for term in d:
        if term == 0:
            continue
        index = vocabulary.setdefault(term, term)
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

print(csr_matrix((data, indices, indptr), dtype=int).toarray())
print(indices)
print(indptr)
print(vocabulary)

[[1 2 2 0 0 0 0 0 0 0 0]
 [0 2 0 2 4 1 1 0 0 0 0]
 [0 1 0 1 1 0 0 0 0 0 0]
 [1 3 0 0 1 1 0 1 1 1 1]
 [1 0 0 0 3 0 0 0 0 0 0]]
[0, 1, 2, 2, 1, 3, 4, 1, 4, 1, 4, 4, 5, 6, 3, 3, 4, 1, 0, 1, 1, 7, 1, 4, 8, 5, 9, 10, 0, 4, 4, 4]
[0, 5, 15, 18, 28, 32]
{1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6, 8.0: 7, 9.0: 8, 10.0: 9, 11.0: 10}


In [16]:
print(csr(X_toy))
print(csr_new(X_toy, site_freq_3users))
X_sparse_toy = csr(X_toy)

[[1 2 2 0 0 0 0 0 0 0 0]
 [0 2 0 2 4 1 1 0 0 0 0]
 [0 1 0 1 1 0 0 0 0 0 0]
 [1 3 0 0 1 1 0 1 1 1 1]
 [1 0 0 0 3 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1 2 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 4 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 3 0 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 3 0 0 0 0 0 0]]


**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [42]:
X_sparse_toy.todense()

AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [18]:
train_data_10users = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), index_col = 'session_id')
train_data_150users = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), index_col = 'session_id')

In [19]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [32]:
X_sparse_10users, a = csr(X_10users)
X_sparse_150users, b = csr(X_150users)

In [44]:
y_10users

array([ 39,  39,  39, ..., 237, 237, 237])

In [22]:
train_data_150users.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,userID
session_id,,,,,,,,,,,
0,1.0,2.0,2.0,1.0,3.0,2.0,4.0,5.0,6.0,7.0,158
1,8.0,2.0,9.0,9.0,10.0,9.0,11.0,12.0,13.0,14.0,158
2,15.0,16.0,9.0,15.0,16.0,13.0,9.0,14.0,16.0,9.0,158
3,9.0,16.0,9.0,9.0,9.0,17.0,9.0,9.0,14.0,13.0,158
4,16.0,15.0,15.0,13.0,14.0,16.0,15.0,16.0,9.0,9.0,158


In [23]:
X_sparse_10users

<14061x4913 sparse matrix of type '<class 'numpy.int64'>'
	with 140574 stored elements in Compressed Sparse Row format>

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [51]:
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

In [24]:


with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)


**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [39]:
site_freq_10users = pickle.load(open(os.path.join(PATH_TO_DATA, 
                       'site_freq_10users.pkl'),"rb"))


In [43]:
site_freq_10users.get('office.microsoft.com')[0]

2

In [40]:
assert a.shape[1] == len(site_freq_10users)

In [26]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

NameError: name 'site_freq_150users' is not defined

## Пути улучшения
В этом проекте свобода творчества на каждом шаге, а 7 неделя проекта посвящена общему описанию (`html`, `ipynb` или `pdf`) и взаимному оцениванию проектов. Что еще можно добавить по первой части проекта:
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- Помимо явного создания разреженного формата можно еще составить выборки с помощью `CountVectorizer`, `TfidfVectorizer` и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle Inclass (желающие могут начать уже сейчас).

На следующей неделе мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 